# Протоколы обмена ключами

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

In [3]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

In [4]:
def randint(a, b):
    """
    Генерация слуайного числа среди заданных.
    """
    
    native_a = int(a)
    native_b = int(b)
    return gmpy2.mpz(random.randint(native_a, native_b))

----

In [5]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime
from Crypto.Random import random

# HKDF и то, что надо для нее
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA512

# Подпись и шифрование
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Cipher import AES

# Математическая библиотека
import gmpy2

# Для генерации идентификаторов
import uuid

# Утилитки
from Crypto.Util.number import long_to_bytes
import binascii

----

## Протокол Диффи-Хеллмана
![Diffe-Hellman Protocol](../images/Diffie-Hellman_4.png)

In [6]:
class DiffieHellmanParams(object):
    def __init__(self, prime_bits: int = 512):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[DiffieHellmanParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [7]:
class DiffieHellmanUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: DiffieHellmanParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        self._sabotage = use_another_value
        
        trace('[DiffieHellmanUser]', f'client {self._id} created')
    
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, отправляю другому пользователю
        # Получаю от него g ** y mod p
        #
        
        x = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{x = }')
        
        intermediate_value = another_client._accept_key_exchange(gmpy2.powmod(self._g, x, self._p))
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[DiffieHellmanUser]', f'[{self._id}]', f'guessed {x = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value) -> int:
        #
        # Получаю g ** x mod p, генерирую у
        #
        
        y = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{y = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return gmpy2.powmod(self._g, y, self._p)
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [8]:
# Параметры протокола
params = DiffieHellmanParams()

[TRACE] [DiffieHellmanParams] p = 11570882813530857704180896782888963760237046712218473278382541378089083982087924496784422297987452167720042313605386970475236342306527558033575371353626157, g = 1583951512341288715042350328286573762588662743852426020893835029226960028040123591080319069454146136215884725045217890704623520125807019498653756239607556


In [9]:
# Два хороших пользователя
alice = DiffieHellmanUser(params)
bob   = DiffieHellmanUser(params)

[TRACE] [DiffieHellmanUser] client 9ed4084c-a105-491c-aca3-586d3c3fedad created
[TRACE] [DiffieHellmanUser] client d8025159-d186-4da4-ac4b-42e0fe3d1d0f created


In [10]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [9ed4084c-a105-491c-aca3-586d3c3fedad] x = mpz(2003856994444373277212620039125739426671271905180875972477183449880647423818369127463695984460284084064091724702339052194391276546648877657650548784020198)
[TRACE] [DiffieHellmanUser] [d8025159-d186-4da4-ac4b-42e0fe3d1d0f] y = mpz(3551519858771118737120761768711551620838161983058102779387742222402701260611171424107451839534270475997599445057456094492388300892813513085384189652585909)
[TRACE] [DiffieHellmanUser] [d8025159-d186-4da4-ac4b-42e0fe3d1d0f] derived key: 393e0212365a3db3ea61d251fc7abe07e6e4966994fa4b29c05f122f7ce275f9
[TRACE] [DiffieHellmanUser] [9ed4084c-a105-491c-aca3-586d3c3fedad] derived key: 393e0212365a3db3ea61d251fc7abe07e6e4966994fa4b29c05f122f7ce275f9


In [11]:
# Теперь врага создам
mallory = DiffieHellmanUser(params, True)

[TRACE] [DiffieHellmanUser] client a79c50ae-7a64-4987-94cd-5fb2c1747fc9 created


In [12]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [a79c50ae-7a64-4987-94cd-5fb2c1747fc9] x = mpz(8491712841675714512865082583920757444812946469480056678170299142961983887175392505560129019566103236800268772677498974136351891036603116252964782088261714)
[TRACE] [DiffieHellmanUser] [d8025159-d186-4da4-ac4b-42e0fe3d1d0f] y = mpz(5848020088349095087009290764529945612109793856159691638042572188890560782369314425082455343586577427857015105362962063949174394018769825545320386517318523)
[TRACE] [DiffieHellmanUser] [d8025159-d186-4da4-ac4b-42e0fe3d1d0f] derived key: 8d3a2a9f20d0d5753a6e3bc4bbebd4f524751da187b855efce62c6a946a80a4b
[TRACE] [DiffieHellmanUser] [a79c50ae-7a64-4987-94cd-5fb2c1747fc9] guessed x = mpz(3772922741610994095902410973831460216790114414325105214028458077512006221559659403300341092896925884979060114631171640396221924196954373040148519527905082)
[TRACE] [DiffieHellmanUser] [a79c50ae-7a64-4987-94cd-5fb2c1747fc9] derived key: f18b8765c6df2fd76b4b40aad877917c7ba608d336e574905b8a4407f054b6cb


----
## Протокол MTI
![Diffe-Hellman Protocol](../images/MTI_4.png)

In [13]:
class MTIParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[MTIParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [14]:
class MTIUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: MTIParams, *,
                 fake_data: gmpy2.mpz = None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = self._generate_group_element()
        self._public_key  = gmpy2.powmod(self._g, self._private_key, self._p) if fake_data is None else fake_data
        
        trace('[MTIUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
        
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому клиенту и получаю от него g ** y mod p
        #
        
        intermediate_value = another_client._accept_key_exchange(m, self.public_key)
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(another_client.public_key, x, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
    
    
    def _accept_key_exchange(self, intermediate_value, public_key):
        #
        # Генерирую у, вычисляю g ** y mod p
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(public_key, y, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(MTIUser.HKDF_DUMMY_STUFF, 
                         MTIUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return m
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [15]:
# Параметры протокола
params = MTIParams(512)

[TRACE] [MTIParams] p = 12506940612787248104964241550668153668505253204772239147241494443493490519217093653799136680482659497324670240146838191471877579664923419828554713751651227, g = 6367079230395177377351774772305759086835397255385798495436991143097384614170440339711069967598048651562392599007054589479742608837625972842960412124034588


In [16]:
# Два хороших пользователя
alice = MTIUser(params)
bob   = MTIUser(params)

[TRACE] [MTIUser] client 8b0fd62a-49b1-4d77-950a-ffd704dec53b created. PK = 11137071348156137440870204924002312908308725560898102432853123960536293267781182626200005585039861997300703084417339235200474407983094754496996121349228347, SK = 2185577648890330299356937514937147693810766143946766937617753162132240171618136047620001936817495794361723225729990561890397105317987376184727975731200001
[TRACE] [MTIUser] client 0afdda4b-aac8-4914-bfdc-9f09c65186a8 created. PK = 5741071645115382158632359897274195800069992157418020138743899304950619420387639018925719640770004383180869697996251609288704242893653836038881724876560924, SK = 855774736568686018435387451291791442417424645455630697699476624619816639271151153552530734871354079589785960429325719644468748053737798541182135902969340


In [17]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [MTIUser] [8b0fd62a-49b1-4d77-950a-ffd704dec53b] x = mpz(4059756797085659763776196486216989732460547754018339753173819733029674317183693666016070527152549777997524400589127506901423700145152883208564621741308688), m = mpz(11355527148244473443368660455833723923062635356658846486616525927097089681090816488067052347316003389487992994259003538995662942086923693553507203358560774)
[TRACE] [MTIUser] [0afdda4b-aac8-4914-bfdc-9f09c65186a8] y = mpz(9272389050219255077244265466240454831729771255705941778251892638115379567505240924872129875359924452635448907017837026542645112733550759860960868462178121), m = mpz(8737190358675064044657851383311567740503339930926204441225655505341535197184736390813144333109347163769597816455406257992430738493160680032153677196319616)
[TRACE] [MTIUser] [0afdda4b-aac8-4914-bfdc-9f09c65186a8] derived key: a97103e24985fa5fdb12b0629598bf86acc9ce6603f6efcf29ec616874592f2b
[TRACE] [MTIUser] [8b0fd62a-49b1-4d77-950a-ffd704dec53b] derived key: a97103e24985fa5fdb12b0

In [18]:
# Теперь врага создам
mallory = MTIUser(params, fake_data=alice.public_key)

[TRACE] [MTIUser] client 20b5c8bb-53b1-4e5a-9826-5615218a4097 created. PK = 11137071348156137440870204924002312908308725560898102432853123960536293267781182626200005585039861997300703084417339235200474407983094754496996121349228347, SK = 5517931674594274624349678077188039894673563637394455202200655343167155692333178414533007636631357394325022329942969695152375117607343761463146208686232365


In [19]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [MTIUser] [20b5c8bb-53b1-4e5a-9826-5615218a4097] x = mpz(1534679556602367844704181616161200028663707760528079144681590868456063708436763522161026082332641309681299626828965533722552745978486661268214661230713295), m = mpz(10199579896757865352509867588193015996797688404101251470013980487455440350450774305986799568347872409776230052248931430555568528046919022531786389875505029)
[TRACE] [MTIUser] [0afdda4b-aac8-4914-bfdc-9f09c65186a8] y = mpz(10597217280941930004153178723717413802841617854410465919765862400618631154837640512789771888656044301218995007099025255731751448155333748652315527540726119), m = mpz(1175279155120468931559279460311865305388261506135818693937707285597467127677679972832087374339266437435194188256071936814705897752808917476402738333356656)
[TRACE] [MTIUser] [0afdda4b-aac8-4914-bfdc-9f09c65186a8] derived key: 541bbb343b6f19df3c57b70f81bf7a2580d493296d0d7bf3ae5fff1c3055cf82
[TRACE] [MTIUser] [20b5c8bb-53b1-4e5a-9826-5615218a4097] derived key: 8ace35989b0a045a183fd

----
## Протокол STS
![STS Protocol](../images/STS_4.png)

In [20]:
class STSParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[STSParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [21]:
class STSUser(object):
    RSA_KEY_LENGTH = 4096
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: STSParams, *, 
                 fake_data=None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = RSA.generate(STSUser.RSA_KEY_LENGTH)
        self._public_key  = self._private_key.publickey() if fake_data is None else fake_data
        
        trace('[STSUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому пользователю, получаю g ** y mod p и зашифрованную подпись
        #
        
        intermediate_value, encrypted_signature = another_client._accept_key_exchange(m)
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Расшифровываю подпись и проверяю ее
        # Если не подошла, то ключ неверный, завершаюсь
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(intermediate_value, m, received_signature, another_client.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return
        
        #
        # Сам теперь подписываю значения и отправляю на проверку другому пользователю
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        if not another_client._check_key_exchange(m, intermediate_value, 
                                                  (cipher.nonce, cipher.encrypt(signature)), 
                                                  self.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Another user refused key exchange')
            self._key = None
            return
        
        #
        # Тут все хорошо
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value):
        #
        # Генерирую у, вычисляю g ** y mod p
        # Вывожу ключ
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Создаю подпись значений и отправляю инициатору
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        encrypted_signature = (cipher.nonce, cipher.encrypt(signature))
        
        return m, encrypted_signature
    
    
    def _check_key_exchange(self, f, s, encrypted_signature, pk):
        #
        # Проверяю подпись от инициатора
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(f, s, received_signature, pk):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return False
        
        #
        # Если все валидно, то ключ верно установлен
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        return True
        
    
    def _sign_values(self, f, s):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        return pss.new(self._private_key).sign(h)
    
    
    def _check_signature(self, f, s, signature, pk):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        verifier = pss.new(pk)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
    
        
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key

-----

In [22]:
# Параметры протокола
params = STSParams(512)

[TRACE] [STSParams] p = 10399801791170482334242562955414187160186497203841080724975007409363453863896509668744273541280583514347750059223613592025443822788376242449959040867826347, g = 7885947742498908289156932149339659772385838188554535395237929933504883784962496453647748380160680710780969309793424492712912952978652095997466681161774590


In [23]:
# Два хороших пользователя
alice = STSUser(params)
bob   = STSUser(params)

[TRACE] [STSUser] client ae50fc79-2f9d-454a-8f1a-58a2361b88d3 created. PK = Public RSA key at 0x114AF1160, SK = Private RSA key at 0x114AD7D90
[TRACE] [STSUser] client 14b56d09-7651-4623-b0eb-3be6beb94c99 created. PK = Public RSA key at 0x114AD72B0, SK = Private RSA key at 0x114AEA220


In [24]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [STSUser] [ae50fc79-2f9d-454a-8f1a-58a2361b88d3] x = mpz(4684501533403193829519983627358608235241032255006882323798366206321333581274576727771498633668457861027803386409270022540390038751242774229022526254330510), m = mpz(15425026169021445988392709818686929702837884727059914682163052343644255949923652892569051146680880522529016433972063775085498658929960407759863941828996)
[TRACE] [STSUser] [14b56d09-7651-4623-b0eb-3be6beb94c99] y = mpz(2670708072206303913633716334991507527820334330370230387591441753899178265694536015458520445203090194904478208774145501183975998180751819498541293655823932), m = mpz(6792054834112659910003650475962410113340460541251948603981114152319653903750853473985351508258072608965093354542299123511646115728351844960379264387587537)
[TRACE] [STSUser] [14b56d09-7651-4623-b0eb-3be6beb94c99] derived key: 924b739da0a785d62f92d17f9c0a73bbb342b086d0905a84087de251627ef39a
[TRACE] [STSUser] [ae50fc79-2f9d-454a-8f1a-58a2361b88d3] derived key: 924b739da0a785d62f92d17f9

In [25]:
# Теперь врага создам
mallory = STSUser(params, fake_data=alice.public_key)

[TRACE] [STSUser] client 40d2a295-4dc7-4cb4-984b-4ae9e403f493 created. PK = Public RSA key at 0x114AF1160, SK = Private RSA key at 0x114AE7790


In [26]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [STSUser] [40d2a295-4dc7-4cb4-984b-4ae9e403f493] x = mpz(5141303679012321521067972648387190148464592980317873505735404617943611501966165482758541215016444293466352817026984894061894172941221142463845307776822566), m = mpz(2379649108113373405170138519396765551362447649378199889954448539444523682006593463034759379287114006520488275698456855239420050137361261103048864842911869)
[TRACE] [STSUser] [14b56d09-7651-4623-b0eb-3be6beb94c99] y = mpz(3726653218925445357668138190911989478281686724711162678659770549592791202217296547556211401318604810109658780841679487859767209111702167980870927609794941), m = mpz(4413368783018545745274785666167946561032315411636817133389787914683034654965508045832872474613600635608687926058738133100183006995175195448081417389800183)
[TRACE] [STSUser] [14b56d09-7651-4623-b0eb-3be6beb94c99] Invalid signature
[TRACE] [STSUser] [40d2a295-4dc7-4cb4-984b-4ae9e403f493] Another user refused key exchange


In [27]:
bob.initiate_key_exchange(mallory)

[TRACE] [STSUser] [14b56d09-7651-4623-b0eb-3be6beb94c99] x = mpz(4110640383328309390210133688308413512384275431042172360410107201295238161084421491180801402094107002197686892852569158038801623399180831456620701077333311), m = mpz(2891912218882188644256927740329565186135007696339125450868347703991153644539257661967390384147921945761181125124585634153170310186158160247130665682305328)
[TRACE] [STSUser] [40d2a295-4dc7-4cb4-984b-4ae9e403f493] y = mpz(5043298490265501048011164128421846983476597044526589807988279823534149755762743804373524838737018605082201039653031616072924486157703559335470108923321979), m = mpz(4869037225523951917272819729258352902781948589413349235609203368719448528852101968710746357921447132464911239611345846651175387989064366445213406093236466)
[TRACE] [STSUser] [14b56d09-7651-4623-b0eb-3be6beb94c99] Invalid signature
